In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from speedy_utils.all import *

setup_logger("D")

DEBUG    | utils_print.py:275 (setup_logger) - Logging set to DEBUG


In [ ]:
setup_logger("D")
def slow_func(x):
    time.sleep(1)  # sleeps for `x` seconds
    if x ==5:
        raise ValueError("Error")
    return x

results = multi_thread(
    func=slow_func,
    orig_inputs=range(10),
    workers=2,
    desc="Demo",
    verbose=True,
    stop_on_error=True,
)

DEBUG    | utils_print.py:275 (setup_logger) - Logging set to DEBUG


Demo:   0%|                                                                                      | 0/10 [00:00<?, ?it/s]

In [ ]:
results

[0, 1, 2, 3, None, RunErr(ValueError), None, None, None, None]

In [ ]:
x = f()

NameError: name 'f' is not defined

In [ ]:
x.terminate()